In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from chromatic import *
import rainbowconnection
import astropy.units as u
from matplotlib import cm
from chromatic import get_phoenix_photons, get_planck_photons, version
import astropy.constants as con
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import emcee
import corner
import speclite as speclite
from matplotlib.artist import Artist
from speclite import filters

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (12, 7),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
plt.style.use('seaborn-deep')

In [ ]:
def quickfit_chisq(parameters, extras, **kwargs):
    
    Tspec = parameters
    rwave, rflux, runc = extras
    
    if Tspec < 2350.:
        Tspec = 2350.
    if Tspec > 5000.:
        Tspec = 5000.
    
    PHOENIX = get_phoenix_photons(temperature=float(Tspec), wavelength = rwave,logg=4.4, metallicity=0.0)[1]
    model = PHOENIX/np.median(PHOENIX)
    chisq = np.nansum((rflux-model)**2/(runc)**2)

    return chisq

In [ ]:
def sigma_clip_rainbow(rainbow, nsigma=5, **kwargs):
    
    #define some of the arrays
    r = rainbow
    unclipped_data = r.flux
    unclipped_err = r.uncertainty
    error_list = [None]*len(r.time)
    function_to_minimize = quickfit_chisq

    #loop through each time point
    for i in tqdm(range(len(r.time))):
        
        initial_parameter_guess = [3750.5]
        extras = [r.wavelength,unclipped_data[:,i],unclipped_err[:,i]]
        results = minimize(function_to_minimize, initial_parameter_guess, extras)
        _Tspec = results['x']
        if _Tspec < 2350.:
            _Tspec = 2350.
        if _Tspec > 5000.:
            _Tspec = 5000.
        print(f'Initial guess was {initial_parameter_guess}')
        print(f'Optimized Tspec is {int(_Tspec)}')
        PHOENIX = get_phoenix_photons(temperature=float(_Tspec), wavelength = r.wavelength,logg=4.4, metallicity=0.0)[1]
        preclip_model = PHOENIX/np.median(PHOENIX)
        print('calculating initial chi^2...')
        chisq = np.nansum((unclipped_data[:,i]-preclip_model)**2/(unclipped_err[:,i])**2)
        unclipped_chisq = chisq/(len(unclipped_data[:,0])-1)
        err_kludge = np.sqrt(unclipped_chisq)
        print(f'initial reduced chi^2: {unclipped_chisq:.1f}')
        print(f'initial error factor: {err_kludge:.1f}')
        print('-----------------')
        print(f'clipping emission lines...')
        points_to_keep = ( (unclipped_data[:,i]-preclip_model)/(unclipped_err[:,i]*err_kludge) < 2.0 )
        r.wavelike["ok"] = points_to_keep
        clipped = r.trim(just_edges=False)
        print('----------------------------------')

        print('calculating new model and chi^2...')
        extras = [clipped.wavelength,clipped.flux[:,i],clipped.uncertainty[:,i]]
        results = minimize(function_to_minimize, initial_parameter_guess, extras)
        Tspec = results['x']
        if Tspec < 2350.:
            Tspec = 2350.
        if Tspec > 5000.:
            Tspec = 5000.
        print(f'Initial guess was {initial_parameter_guess}')
        print(f'Optimized Tspec is {int(Tspec)}')
        PHOENIX = get_phoenix_photons(temperature=float(Tspec), wavelength = clipped.wavelength,logg=4.4, metallicity=0.0)[1]
        model = PHOENIX/np.median(PHOENIX)
        chisq = np.nansum((clipped.flux[:,i]-model)**2/(clipped.uncertainty[:,i])**2)
        reduced_chisq = chisq/(len(clipped.wavelength)-1)
        err_kludge = np.sqrt(reduced_chisq)
        print(f'updated reduced chi^2: {reduced_chisq:.1f}')
        print(f'error factor: {err_kludge:.1f}')
        print('-----------------')
        print(f'clipping {nsigma}-sigma outliers...')
        points_to_keep = ( np.abs(clipped.flux[:,i]-model)/(clipped.uncertainty[:,i]*err_kludge) <= nsigma )
        clipped.wavelike["ok"] = points_to_keep
        clipped = clipped.trim(just_edges=False)
        print('----------------------------------')
        
        print('calculating another model and chi^2...')
        extras = [clipped.wavelength,clipped.flux[:,i],clipped.uncertainty[:,i]]
        results = minimize(function_to_minimize, initial_parameter_guess, extras)
        Tspec = results['x']
        if Tspec < 2350.:
            Tspec = 2350.
        if Tspec > 5000.:
            Tspec = 5000.
        print(f'Initial guess was {initial_parameter_guess}')
        print(f'Optimized Tspec is {int(Tspec)}')
        PHOENIX = get_phoenix_photons(temperature=float(Tspec), wavelength = clipped.wavelength,logg=4.4, metallicity=0.0)[1]
        model = PHOENIX/np.median(PHOENIX)
        chisq = np.nansum((clipped.flux[:,i]-model)**2/(clipped.uncertainty[:,i])**2)
        reduced_chisq = chisq/(len(clipped.wavelength)-1)
        err_kludge = np.sqrt(reduced_chisq)
        print(f'updated reduced chi^2: {reduced_chisq:.1f}')
        print(f'error factor: {err_kludge:.1f}')
        print('-----------------')
        print(f'clipping points with error > 20% ...')
        err_outliers = ( clipped.uncertainty*err_kludge < 0.2 )
        clipped.wavelike["ok"] = err_outliers[:,i]
        final = clipped.trim(just_edges=False)
        print('----------------------------------')
        
        print('calculating a (hopefully final) model and chi^2...')
        extras = [final.wavelength,final.flux[:,i],final.uncertainty[:,i]]
        results = minimize(function_to_minimize, initial_parameter_guess, extras)
        Tspec = results['x']
        if Tspec < 2350.:
            Tspec = 2350.
        if Tspec > 5000.:
            Tspec = 5000.
        print(f'Initial guess was {initial_parameter_guess}')
        print(f'FINAL Optimized Tspec is {int(Tspec)}')
        PHOENIX = get_phoenix_photons(temperature=float(Tspec), wavelength = final.wavelength,logg=4.4, metallicity=0.0)[1]
        finalmodel = PHOENIX/np.median(PHOENIX)
        chisq = np.nansum((final.flux[:,i]-finalmodel)**2/(final.uncertainty[:,i])**2)
        reduced_chisq = chisq/(len(final.wavelength)-1)
        err_kludge = np.sqrt(reduced_chisq)
        error_list[i] = err_kludge
        final.uncertainty = final.uncertainty*err_kludge
        print(f'updated reduced chi^2: {reduced_chisq:.1f}')
        print(f'error factor: {err_kludge:.1f}')
        print('----------------------------------')

        sanity_check = np.nansum((final.flux[:,i]-finalmodel)**2/(final.uncertainty[:,i]*err_kludge)**2)
        should_be_1 = sanity_check/(len(final.wavelength)-1)
        ##########################################

        print('plotting...')
        label=f'Order {order} | BJD {r.time[i].value:.1f} | err kludge: {error_list[i]:0.1f} | reduced chisq: {should_be_1:.2f}'
        fig, axs = plt.subplots(2,2,figsize=(12,10),sharex=True)
        fig.suptitle(label,fontsize=20)

        ax1 = axs[0,0]
        ax2 = axs[1,0]
        ax3 = axs[0,1]
        ax4 = axs[1,1]

        # Upper left plot, the pre-clipped data + model
        ax1.errorbar(r.wavelength.value,
                     unclipped_data[:,i],
                     yerr=unclipped_err[:,i],color='red',alpha=0.7,
                     zorder=-10,label='pre-clipped data',fmt='',capsize=0)
        ax1.plot(r.wavelength.value,preclip_model,alpha=1,
                     color='black',zorder=100,label=f'{int(_Tspec)} K PHOENIX model')
        ax1.set_title('Pre-clipped',fontsize=16)
        ax1.set_ylim(0,1.5)
        ax1.set_ylabel('normalized flux',fontsize=16)
        ax1.legend(loc='lower right')

        # Lower left plot, residuals on original data and model
        ax2.errorbar(r.wavelength.value,
                     (unclipped_data[:,i]-preclip_model)/unclipped_err[:,i],
                     yerr=unclipped_err[:,i],color='black',fmt='',capsize=0,zorder=100)
        #ax2.set_ylim(-nsigma,nsigma)
        ax2.set_xlim(r.wavelength[0].value,r.wavelength[-1].value)
        ax2.set_xlabel(r'Wavelength ($\mu$)',fontsize=16)
        ax2.set_ylabel(r'Residual ($\sigma$)',fontsize=16)
        ax2.set_title('Residuals',fontsize=16)
        ax2.set_ylim(-10,10)
        ax2.axhspan(-1,1,color='red',alpha=0.2,zorder=10)
        ax2.axhspan(-2,2,color='green',alpha=0.2,zorder=0)
        ax2.axhspan(-3,3,color='gray',alpha=0.3,zorder=-10)

        # Upper right plot, the clipped data + model
        ax3.errorbar(final.wavelength.value,
                     final.flux[:,i],yerr=final.uncertainty[:,i],alpha=0.7,
                     color='teal',zorder=10,label='post-clipped data',fmt='',capsize=0)
        ax3.set_title('Post-clipped',fontsize=16)
        ax3.set_ylim(0,1.5)
        ax3.plot(final.wavelength.value,finalmodel,alpha=1,
                     color='black',zorder=100,label=f'{int(Tspec)} K PHOENIX model')
        ax3.legend(loc='lower right')

        # Lower right plot, the final residuals
        ax4.errorbar(final.wavelength.value,
                 (final.flux[:,i]-finalmodel)/final.uncertainty[:,i],
                 yerr=final.uncertainty[:,i],color='black',fmt='',capsize=0,zorder=100)
        ax4.set_ylim(-nsigma,nsigma)
        ax4.set_xlim(r.wavelength[0].value,r.wavelength[-1].value)
        ax4.set_xlabel(r'Wavelength ($\mu$)',fontsize=16)
        ax4.set_title('Residuals',fontsize=16)
        ax4.axhspan(-1,1,color='red',alpha=0.2,zorder=10)
        ax4.axhspan(-2,2,color='green',alpha=0.2,zorder=0)
        ax4.axhspan(-3,3,color='gray',alpha=0.3,zorder=-10)
        #ax4.legend(loc='upper right')
        plt.savefig(f'figs/nres_sigmaclip/{visit}_order{order}_{i}_sigmaclip_timespec.pdf')
        plt.show()
        
    return error_list, final

In [ ]:
def initialize(order = 53,
               visit = 'fall',
               vshift=-8.7,
               dw=0.05*u.nm):
    """
    Load in and process NRES spectra
    """
    
    #define visit-specific parameters (why not use dicts? good question)
    if visit == 'fall':
        data_path = 'data/NRES/fall_2021_spectra/*e92-1d.fits.fz'
        phot_amplitudes = np.array([0.054,0.067,0.049])
        phot_errs = np.array([0.006,0.006,0.004])
        phase = 2.992 - np.pi/2.
        T0 = (59455.48 + 2400000.5)
        edges = np.array([2459451.0,2459452.0,2459454.0,2459457.0,2459459.0,2459460.0,2459461.0])*u.day
        _ntimes = len(edges)
        
    if visit == 'spring':
        data_path = 'data/NRES/spring_2022_spectra/*e92-1d.fits.fz'
        phot_amplitudes = np.array([0.07,0.071])
        phot_errs = np.array([0.004,0.004])
        phase = 2.87 - np.pi/2.
        T0 = (59455.48 + 2400000.5) + (27.*8.46)
        edges = np.array([2459680.5,2459681.5,2459682.5,
                          2459683.5,2459684.5,2459685.5,
                          2459686.5,2459687.5,2459688.5])*u.day
        _ntimes = len(edges)
    
    #create the rainbow object and apply the velocity shift
    r = Rainbow(data_path, format='nres', order=order)
    shifted_r = r.align_wavelengths().shift(vshift *u.km/u.s)

    #correct for atmospheric transmission from skycalc outputs
    transmission_data = np.loadtxt('data/transmission_comp.dat')
    mol_wave = transmission_data[:,0]*(1e-3) * u.micron
    mol_data = transmission_data[:,1]
    this_orders_tellurics = bintogrid(mol_wave.value,mol_data,newx=shifted_r.wavelength.value,visualize=False)
    transparent_wavelengths = (this_orders_tellurics['y'] > 0.995)
    shifted_r.wavelike["ok"] = transparent_wavelengths

    #bin the rainbow so the clipping doesn't take a thousand years
    preclipped_rainbow = shifted_r.trim(just_edges=False).bin(dw=dw,time_edges = edges,ntimes = _ntimes,
                                                                 minimum_points_per_bin=1)

    #run the sigma clipping function to return a final processed rainbow
    this_orders_errbar,processed_r = sigma_clip_rainbow(rainbow=preclipped_rainbow,nsigma=3)
    
    return processed_r, phot_amplitudes, phot_errs, phase, T0, this_orders_errbar,this_orders_tellurics

In [ ]:
def semi_amplitude_model(parameters=[0.15,0.06,3000,3800],
                         plot=False,
                         visit='fall',
                         **kwargs):
    
    bandpass=np.linspace(3500.,8500.,300)*u.angstrom
    sdss_responses = speclite.filters.load_filters('sdss2010-g','sdss2010-r','sdss2010-i')
    response_g = sdss_responses[0].interpolator(bandpass)
    response_r = sdss_responses[1].interpolator(bandpass)
    response_i = sdss_responses[2].interpolator(bandpass)

    f_spot,df_spot,T_spot,T_amb = parameters

    ds_spot = get_phoenix_photons(temperature=float(T_spot), wavelength = bandpass,logg=4.4, metallicity=0.0)[1]
    ds_amb = get_phoenix_photons(temperature=float(T_amb), wavelength = bandpass,logg=4.4, metallicity=0.0)[1]
    d_lambda = (bandpass[1]-bandpass[0])

    ds_over_s = -df_spot*((1.-ds_spot/ds_amb) / (1.-f_spot*(1.-ds_spot/ds_amb)))
    semi_amplitude = np.abs(ds_over_s)

    numerator = np.nansum(semi_amplitude*response_g*d_lambda)
    denominator = np.nansum(response_g*d_lambda)
    gp = numerator/denominator

    numerator = np.nansum(semi_amplitude*response_r*d_lambda)
    denominator = np.nansum(response_r*d_lambda)
    rp = numerator/denominator

    numerator = np.nansum(semi_amplitude*response_i*d_lambda)
    denominator = np.nansum(response_i*d_lambda)
    ip = numerator/denominator

    if visit == 'spring':
        model = np.array([gp,rp])
        model_coords = [4750,6200]
    else:
        model = np.array([gp,rp,ip])
        model_coords = [4750,6200,7550]

    if plot:
        fig, ax1 = plt.subplots(figsize=(14,8))
        chisq = np.nansum((photometry_amplitudes-model)**2/(photometry_errs**2))
        reduced_chisq = chisq/(len(photometry_amplitudes)-1)
        kludge = np.sqrt(reduced_chisq)
        title_label=f'Order {order} | fspot={f_spot:.2f}+/-{df_spot:.2f}| Tspot={int(T_spot/10)*10} | Tamb={int(T_amb/10)*10} | err kludge: {kludge:0.1f} | reduced chisq: {reduced_chisq:.2f}'
        fig.suptitle(title_label,fontsize=20)

        ax1.set_xlabel(r'Wavelength $\AA$',fontsize=20)
        ax1.set_ylabel(r'$\frac{\Delta S(\lambda)}{S_{avg}}$',fontsize=16)
        ax1.plot(bandpass, semi_amplitude,zorder=100)
        ax1.scatter(model_coords,model,color='red',zorder=100,label='model value')
        ax1.errorbar(model_coords,photometry_amplitudes,photometry_errs,color='k',
                     label='Measured Variability',fmt='o')
        ax1.set_xlim(3500,8500)
        ax1.set_ylim(0,0.1)
        ax1.legend(loc='upper right')

        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        ax2.fill_between(bandpass.value, bandpass.value*0, response_g,color='orange',
                         zorder=-100,label='SDSS g filter response',alpha=0.3)
        ax2.fill_between(bandpass.value, bandpass.value*0, response_r,color='teal',
                         zorder=-100,label='SDSS r filter response',alpha=0.3)
        ax2.fill_between(bandpass.value, bandpass.value*0, response_i,color='purple',
                         zorder=-100,label='SDSS i filter response',alpha=0.3)
        ax2.set_ylabel('Filter Response',fontsize=16)
        ax2.set_ylim(0,0.6)
        ax2.legend(loc='lower right')

        plt.savefig(f'figs/photometric_variability/bestfits/{visit}_order{order}_bestfit_photovariability_timespec.pdf')
        plt.show()

    return model

In [ ]:
def avg_spectrum_model(parameters=[0.15,0.06,3000,3800],
                       plot=False,
                       **kwargs):

    f_spot,df_spot,T_spot, T_amb = parameters
    combined = f_spot*spot_spec + (1.-f_spot)*amb_spec
    combined_spec = combined/np.nanmedian(combined)
    
    chisq = np.nansum((nres_avg_1dspec-combined_spec)**2/(nres_avg_1derr**2))
    reduced_chisq = chisq/(len(nres_avg_1dspec)-4)
    kludge = np.sqrt(reduced_chisq)
    
    if plot:
        title_label=f'Order {order} | fspot={f_spot:.2f}+/-{df_spot:.2f}| Tspot={int(T_spot/10)*10} | Tamb={int(T_amb/10)*10} | err kludge: {kludge:0.1f} | reduced chisq: {reduced_chisq:.2f}'
      
        #now set up the figure(s)
        fig, [ax0,ax1,ax2] = plt.subplots(3,1,figsize=(12,18),sharex=True)
        fig.suptitle(title_label,fontsize=18)
        
        ax0.plot(data_wave, combined_spec, label = 'Spot model',color='k',alpha=0.9,zorder=100)
        ax0.errorbar(data_wave, nres_avg_1dspec, yerr=nres_avg_1derr, zorder=-1000,color='teal',label='NRES time-averaged spectrum',fmt='',alpha=0.9)
        ax0.set_ylim(0.5,1.5)
        ax0.set_ylabel('Relative Flux',fontsize=16)
        ax0.legend(loc='lower right')

        ax1.errorbar(data_wave,(nres_avg_1dspec-combined_spec)/(nres_avg_1derr),
                     yerr=(nres_avg_1derr),label='Residual',color='k',zorder=100)
        ax1.set_ylabel(r'$\sigma$',fontsize=16)
        ax1.set_ylim(-5,5)
        ax1.legend(loc='lower right')
        ax1.axhspan(-1,1,color='red',alpha=0.2,zorder=10)
        ax1.axhspan(-2,2,color='green',alpha=0.2,zorder=0)
        ax1.axhspan(-3,3,color='gray',alpha=0.3,zorder=-10)
        
        ax2.plot(this_orders_tellurics['x'], this_orders_tellurics['y'],label='Telluric Spectrum')
        ax2.axhline(0.995,color='red',label='0.995 cutoff')
        ax2.set_xlabel(r'Wavelength ($\mu$)',fontsize=16)
        ax2.set_ylabel('Molecular Transmission Fraction',fontsize=16)
        ax2.legend(loc='upper right')
        ax2.set_ylim(0.95,1.01)
        ax2.set_yscale('log')
        ax2.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])

        plt.savefig(f'figs/avg_spectra_models/bestfits/{visit}_order{order}_bestfit_avgspectrum_timespec.pdf')

    return combined_spec,chisq

In [ ]:
def time_spectrum_model(parameters=[0.15,0.06,3000,3800],
                        T_rot = 4.86, # days
                        plot=False,
                        **kwargs):

    f_spot,df_spot,T_spot, T_amb = parameters

    combined = f_spot*spot_spec + (1.-f_spot)*amb_spec
    S_avg_model = combined/np.median(combined)
    contrast = (1.-spot_spec/amb_spec)
    dS_over_S = np.abs(-df_spot*( contrast / (1.-f_spot*(contrast))))

    spec_model = np.ones_like(nres_rainbow.flux)
    nres_spec_comparison = np.ones_like(nres_rainbow.flux)

    for i in range(len(data_times)):
        S_t_model = 0.0
        S_t_model = S_avg_model * ( 1. + dS_over_S * np.cos(2.0*np.pi*data_times[i].value/T_rot + phase) )
        spec_model[:,i] = S_t_model/np.median(S_t_model)
        nres_spec_comparison[:,i] = nres_rainbow.flux[:,i]/nres_avg_1dspec
        
    chisq = np.nansum((nres_rainbow.flux - spec_model)**2/(nres_rainbow.uncertainty)**2)
    reduced_chisq = chisq/(len(nres_rainbow.wavelength)*len(data_times)-4)
    kludge = np.sqrt(reduced_chisq)

    if plot:

        model_data_times = np.linspace(data_times[0].value-1,data_times[-1].value+1,1000)
        cmap = cm.get_cmap('cividis', len(data_times))
        colormap = cmap(np.linspace(0, 1, len(data_times)))

        fig, axs = plt.subplots(2,3,figsize=(18,10))
        title_label=f'Order {order} | fspot={f_spot:.2f}+/-{df_spot:.2f}| Tspot={int(T_spot/10)*10} | Tamb={int(T_amb/10)*10} | err kludge: {kludge:0.1f} | reduced chisq: {reduced_chisq:.2f}'
        fig.suptitle(title_label,fontsize=20)

        ax1 = axs[0,0]
        ax2 = axs[1,0]
        ax3 = axs[0,1]
        ax4 = axs[1,1]
        ax5 = axs[0,2]
        ax6 = axs[1,2]

        for i in range(len(data_times)):
            ax3.errorbar(data_wave,nres_rainbow.flux[:,i],yerr=nres_rainbow.uncertainty[:,i],alpha=0.6,
                         color=colormap[i],fmt='')
            ax4.plot(data_wave,(spec_model[:,i]-nres_rainbow.flux[:,i])/nres_rainbow.uncertainty[:,i],
                     alpha=0.6,color=colormap[i])
        
        ax1.plot(data_wave,(spot_spec/np.median(spot_spec)-amb_spec/np.median(amb_spec))/(T_amb-T_spot),color='k')
        ax1.set_title(r'Spot model dS/dT',fontsize=16)
        #ax1.set_ylim(0.85,1.15)
        ax1.set_xlabel(r'Wavelength ($\mu$)',fontsize=16)
        ax1.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        
        ax2.plot(data_wave,dS_over_S)
        ax2.set_title(r'Spot model dS/S',fontsize=16)
        ax2.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])

        ax3.plot(data_wave, np.median(spec_model,axis=1),color='k',zorder=10,label='median model spectrum')
        ax3.set_title('NRES data',fontsize=16)
        ax3.set_ylim(0.5,1.5)
        ax3.set_xlabel(r'Wavelength ($\mu$)',fontsize=16)
        ax3.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        ax3.legend()
           
        ax4.set_title(r'Residuals ',fontsize=16)
        ax4.set_xlabel(r'Wavelength ($\mu$)',fontsize=16)
        ax4.set_ylabel(r'$\sigma$',fontsize=16)
        ax4.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])

        speccompplot = ax5.imshow(np.transpose(nres_spec_comparison),aspect='auto',cmap='Reds')
        ax5.set_title(r'NRES S($\lambda$,t)/S($\lambda$)',fontsize=16)
        ax5.set_ylabel('Observation #',fontsize=16)
        ax5.set_xlabel('Wavelength array #',fontsize=16)
        plt.colorbar(speccompplot, ax = ax5)
        
        specplot= ax6.imshow(np.transpose(spec_model),aspect='auto',cmap='Reds')
        ax6.set_title('PHOENIX Model Spectra',fontsize=16)
        ax6.set_ylabel('Observation #',fontsize=16)
        ax6.set_xlabel('Wavelength array #',fontsize=16)
        plt.colorbar(specplot, ax = ax6) 

        plt.savefig(f'figs/time_domain_spectra_models/bestfits/{visit}_order{order}_bestfit_timespectra_timespec.pdf')
        plt.show()

    return spec_model,chisq

In [ ]:
def lnprob(parameters=[0.15,0.06,3000,3800],
           plot = False, **kwargs):

    f_spot,df_spot,T_spot,T_amb = parameters

    if (0.0<=f_spot<=0.5) and (0.<=df_spot<f_spot) and (2300.0<=T_spot<=4000.0) and (2300.0<=T_amb<=4500.0):

        ln_like=0

        spot_spec = get_phoenix_photons(temperature=float(T_spot), wavelength = data_wave,logg=4.4, metallicity=0.0)[1]
        amb_spec = get_phoenix_photons(temperature=float(T_amb), wavelength = data_wave,logg=4.4, metallicity=0.0)[1]

        "S_t calculations"
        S_t_model,chisq_time_spec = time_spectrum_model(parameters,plot=plot)
        ln_like += (np.nansum(1/np.sqrt(2*np.pi*(nres_rainbow.uncertainty))) - 0.5*chisq_time_spec)

    else:
        ln_like = -np.inf

    return ln_like

def negative_lnprob(parameters,**kwargs):
    return -lnprob(parameters)

In [ ]:
def do_mcmc(order=53,
            nsteps=1000,
            visit='fall',
            vshift=None,
            dw=0.05*u.nm,
            **kwargs):
    
    variables =  initialize(order=order,visit=visit,vshift=vshift,dw=dw)
    nres_rainbow = variables[0]
    photometry_amplitudes = variables[1]
    photometry_errs = variables[2]
    phase = variables[3]
    T0 = init_vars[4]
    this_orders_errbar = variables[5]
    this_orders_tellurics = variables[6]
    
    _1dof = len(nres_rainbow.wavelength)-4
    _2dof = len(nres_rainbow.wavelength)*len(nres_rainbow.time)-4
    init_st_model, time_spec_chisq = time_spectrum_model(visit=visit)
    time_spec_kludge = np.sqrt(time_spec_chisq/(_2dof))
    init_savg_model, avg_spec_chisq = avg_spectrum_model(visit=visit)
    avg_spec_kludge = np.sqrt(avg_spec_chisq/(_1dof))
    print(f'multipling avg spectrum uncertainty by {avg_spec_kludge} and the 2d uncertainty array by {time_spec_kludge}')
    
    nres_avg_1dspec = nres_rainbow.get_average_spectrum()
    nres_avg_1derr = np.nanmedian(nres_rainbow.uncertainty,axis=1)*avg_spec_kludge
    nres_rainbow.uncertainty = nres_rainbow.uncertainty*time_spec_kludge
    data_times = nres_rainbow.timelike['time']
    data_wave = nres_rainbow.wavelength

    # intialize some walkers
    ndim, nwalkers = 4, 100
    burnin = int(0.3*nsteps)
    
    fitted_parameters=[0.15,0.06,3000,3800]
    # these are initial parameters
    fspot_init = np.random.uniform(fitted_parameters[0]-0.1, fitted_parameters[0]+0.1, nwalkers)
    dfspot_init = np.random.uniform(fitted_parameters[1]-0.05, fitted_parameters[1]+0.05, nwalkers)
    Tspot_init = np.random.uniform(fitted_parameters[2]-300, fitted_parameters[2]+300, nwalkers)
    Tamb_init = np.random.uniform(fitted_parameters[3]-100, fitted_parameters[3]+100, nwalkers)

    p0 = np.transpose([fspot_init, dfspot_init, Tspot_init, Tamb_init])

    # set up file saving for the samples when finished
    filename = f"data/{visit}_order{order}_{nsteps}steps_samples_timespec.h5"
    backend = emcee.backends.HDFBackend(filename)
    backend.reset(nwalkers, ndim)

    # Initialize the sampler
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, backend=backend)

    # run the sampler
    result = sampler.run_mcmc(p0, nsteps)

    samples = sampler.chain[:, burnin:, :].reshape((-1, ndim)).T

    fspot_sam, dfspot_sam, Tspot_sam, Tamb_sam = samples

    sig1_fspot = np.percentile(fspot_sam, [16., 50., 95.])
    sig1_dfspot = np.percentile(dfspot_sam, [16., 50., 95.])
    sig1_Tspot = np.percentile(Tspot_sam, [16., 50., 95.])
    sig1_Tamb = np.percentile(Tamb_sam, [16., 50., 95.])

    # Define the 50% percentile and 1-sigma interval parameter values from the samples
    best_params = np.array([sig1_fspot[1], sig1_dfspot[1], sig1_Tspot[1], sig1_Tamb[1]])
    best_params_err_lower = best_params-[sig1_fspot[0], sig1_dfspot[0], sig1_Tspot[0], sig1_Tamb[0]]
    best_params_err_higher = [sig1_fspot[2], sig1_dfspot[2], sig1_Tspot[2], sig1_Tamb[2]]-best_params
    variable_names = [r'f$_{\rm{spot}}$',r'df$_{\rm{spot}}$',r'T$_{\rm{spot}}$',r'T$_{\rm{amb}}$']
    normie_names = ['fspot','dfspot','Tspot','Tamb']
    
    # Print out the results
    for i in range(ndim):
        print(f'{normie_names[i]} = {best_params[i]:.2f} + {best_params_err_higher[i]:.2f} - {best_params_err_lower[i]:.2f}')
    final_avg_Teff = (best_params[0]*best_params[2]**4 + (1-best_params[0])*best_params[3]**4)**(1/4)
    print('Teff = ',int(final_avg_Teff/10)*10)

    #make the corner plot
    figure = corner.corner(samples.T, labels=variable_names,
                           quantiles=[0.16, 0.5, 0.84],
                           show_titles=True, title_kwargs={"fontsize": 16})

    # Extract the axes, loop through to set all the limits
    axes = np.array(figure.axes).reshape((ndim, ndim))
    for yi in range(ndim):
        ax = axes[yi, 0]
        ax.set_xlim(0,0.5)
        ax = axes[yi, 1]
        ax.set_xlim(0,0.25)
        ax = axes[yi, 2]
        ax.set_xlim(2300,4000)
        ax = axes[yi, 3]
        ax.set_xlim(3600,4500)
    plt.savefig(f'figs/corner_plots/{visit}_order{order}_corner_timespec.pdf')
    
    # Plot models
    semi_amplitude_model(parameters = best_params, plot=True)
    avg_spectrum_model(parameters = best_params, plot=True)
    time_spectrum_model(parameters=best_params, plot = True)
    
    # Plot samples
    fig, axs = plt.subplots(2,2,figsize=(10,10))
    ax1 = axs[0,0]
    ax2 = axs[1,0]
    ax3 = axs[0,1]
    ax4 = axs[1,1]
    #plot dfspot vs fspot
    ax1.scatter(dfspot_sam,fspot_sam,c=fspot_sam,cmap='BrBG',alpha=0.6,edgecolor=None)
    ax1.plot(dfspot_sam,dfspot_sam)
    ax1.set_ylabel(r'f$_{\rm{spot}}$',fontsize=20)
    ax1.set_ylim(0,0.5)
    ax1.set_xlim(0,0.25)
    #plt.colorbar(p1,ax=ax1)
    
    #plot dfspot vs log(Tspot/Tamb)
    ax2.scatter(dfspot_sam,np.log(Tspot_sam/Tamb_sam),c=np.log(Tspot_sam/Tamb_sam),
                cmap='BrBG',alpha=0.6,edgecolor=None)
    ax2.set_xlabel(r'$\Delta$ f$_{spot}$',fontsize=20)
    ax2.set_ylabel(r'Log(T$_{\rm{spot}}$/T$_{\rm{amb}}$)',fontsize=20)
    ax2.set_xlim(0,0.25)
    ax2.set_ylim(-0.5,0)
    #plt.colorbar(p2,ax=ax2)
    
    #plot fspot vs log(N), the number of spots
    ax3.scatter(fspot_sam,np.log((fspot_sam/dfspot_sam)**2),c=np.log(Tspot_sam/Tamb_sam),
                cmap='BrBG',alpha=0.6,edgecolor=None)
    ax3.set_ylim(0,6)
    ax3.set_xlim(0,0.5)
    ax3.set_ylabel(r'Log(N$_{\rm{spot}}$)',fontsize=20)
    #ax3.set_yscale('log')
    #plt.colorbar(p3,ax=ax3)
    
    #plot fspot vs log(f1), the filling factor of 1 spot
    ax4.scatter(fspot_sam,np.log(dfspot_sam**2/fspot_sam),c=np.log(Tspot_sam/Tamb_sam),
                cmap='BrBG',alpha=0.2,edgecolor=None)
    ax4.set_xlabel(r'f$_{\rm{spot}}$',fontsize=20)
    ax4.set_xlim(0,0.5)
    ax4.set_ylim(-6,0)
    #ax4.set_yscale('log')
    ax4.set_ylabel(r'Log(f$_1$)',fontsize=20)
    #plt.colorbar(p4,ax=ax4)
    
    plt.savefig(f'figs/samples/{visit}_order{order}_plotsamples_timespec.png',dpi=500)

In [ ]:
#Set some params for model initialization
order = 65
visit = 'fall'
vshift = -8.7
dw = 0.05 * u.nm

init_vars =  initialize(order=order,visit=visit,vshift=vshift,dw=dw)
nres_rainbow = init_vars[0]
photometry_amplitudes = init_vars[1]
photometry_errs = init_vars[2]
phase = init_vars[3]
T0 = init_vars[4]
this_orders_errbar = init_vars[5]
this_orders_tellurics = init_vars[6]

#define some global variables
data_times = nres_rainbow.timelike['time']
data_wave = nres_rainbow.wavelength
nres_avg_1dspec = nres_rainbow.get_average_spectrum()
nres_avg_1derr = np.nanmedian(nres_rainbow.uncertainty,axis=1)*2
spot_spec = get_phoenix_photons(temperature=3000., wavelength = data_wave,logg=4.4, metallicity=0.0)[1]
amb_spec = get_phoenix_photons(temperature=3800., wavelength = data_wave,logg=4.4, metallicity=0.0)[1]

In [ ]:
# do_mcmc(order = order,
#     nsteps = 10,
#     visit=visit,
#     vshift=-8.7,
#     dw=dw)

In [ ]:
for i in tqdm(range(53,90)):
    order = int(i)
    do_mcmc(order = order,
            nsteps = 1000,
            visit='fall',
            vshift=-8.7,
            dw=0.05*u.nm)

In [ ]:
for i in tqdm(range(53,90)):
    do_mcmc(order = int(i),
            nsteps = 1000,
            visit='spring',
            vshift=35.,
            dw=0.05*u.nm)